تصميم آلية لحل النزاعات في نظام خبير استشاري إعلامي


## السيناريو المقترح



يُدير نظام خبير استشاري إعلامي بيانات من مصادر متعددة لتقديم توصيات حول الأخبار التي يجب نشرها أو الأولويات التحريرية
تضارب القواعد قد يحدث عندما

تُظهر قاعدة أن المصدر "أ" موثوق بناءً على تاريخ ماضي.
قاعدة أخرى تشير إلى أن المصدر نفسه يفتقد الموثوقية بسبب تقرير جديد غير مثبت.




### آلية حل النزاع المقترحة


1. الطرق المستخدمة لحل النزاع

الأولوية (Priority): القواعد ذات الأولوية العليا تُطبَّق أولاً.

التوقيت (Recency): يتم إعطاء الأفضلية للقواعد التي تعتمد على معلومات أكثر حداثة.

التخصص (Specificity): القواعد الأكثر تحديدًا لحالة معينة تتفوق على القواعد العامة.

2. مبررات اختيار الطريقة

الأولوية: تُساعد في إعطاء ثقل أكبر للقواعد الأكثر أهمية بالنسبة للنظام (مثل قواعد السلامة أو النزاهة الإعلامية).

التوقيت: يضمن أن النظام يعتمد على أحدث المعلومات المتاحة.

التخصص: يسمح للنظام بالتكيف مع الحالات الفريدة بدلًا من استخدام القواعد العامة.

## كيفية حل النزاع بين القواعد 



يتم ترتيب القواعد حسب الأولوية (الأعلى أولاً).

في حال تساوي الأولوية، تُستخدم التوقيت كمعيار.

إذا استمرت المساواة، تُفَضَّل القاعدة ذات التخصص الأعلى.

### ترقية الآلية بالاعتماد على الكود المرفق


In [172]:
from experta import Rule , KnowledgeEngine ,Fact
from datetime import datetime, timedelta


In [173]:
class Credibility(Fact): #  لموضوع الاول عن المصداقية
    """Represents the environment a trainee interacts with."""
    pass

class Content(Fact):   # الموضوع الثاني عن المحتوي
    """Represents the job type for the trainee."""
    pass


class Interaction(Fact): # الموضوع الثالث عن التفاعل
    """Indicates whether feedback is required."""
    pass

class Medium(Fact): # الموضوع الرابع 
    """Represents the medium selected based on rules.""" # يمثل الوسيط الذي تم اختياره بناءً على القواعد 
    pass


class MediaAdvisor(KnowledgeEngine):  # الكلاس الرئيسي الذي بداخله يتم تعريف كل القواعد 
    def __init__(self):
        super().__init__()
        print("Initializing the Media Advisor Expert System...\n")

          # ----------------------------------  Rules about Credibility   ------------------

# Rule 1                  القاعدة الاولي تشير الي ان المصدر موثوق مئة بالمئة 
    @Rule(Credibility(credibility='Old person') |
          Credibility(credibility='Ancient history')|
                    Credibility(credibility='Credibility') )

    def Displays(self): # يعرض
        print("Rule triggered: Credibility is 'Displays'.")
        self.declare(Fact(stimulus_situation='Displays'))

            
# Rule 2  القاعدة الثالثنية اذا كان المصدر موثوق بنسبة 80% فما فوق يعتمد
    @Rule(Credibility(credibility=MATCH.credibility),
         TEST(lambda credibility: credibility.isdigit() and int(credibility) >= 80))
    def HighCredibility(self):
        print("Rule triggered: High credibility detected.")
        self.declare(Fact(stimulus_situation='depends'))

 # Rule 3
    @Rule(Credibility(credibility='Underexcited') ) # غير مثيت 

    def Away(self): # يبعد
        print("Rule triggered: Credibility is 'Away'.")
        self.declare(Fact(stimulus_situation='Away'))



        # ----------------------------------  Rules about Content   --------------------------

# Rule 4
    @Rule(Content(content='National') |
      Content(content='Security'))
    def CheckContent(self):
        print("Rule triggered: Content requires 'Check'.")
        self.declare(Fact(stimulus_response='Check'))


# Rule 5  قاعدة جديدة: إذا كان المحتوى مثيرًا للجدل، يتم تنبيه المسؤول
    @Rule(Content(content='controversial') |
      Content(content='sensitive') |
      Content(content='violent'))
    def AlertAdmin(self):
        print("Rule triggered: Content is 'Controversial'. Notifying the admin.")
        self.declare(Fact(stimulus_response='Alert'))

        # قاعدة: تحديد إذا كان المحتوى حديثًا
# Rule 6
    @Rule(Content(content=MATCH.content, updated_at=MATCH.updated_at),
      TEST(lambda updated_at: (datetime.now() - updated_at).total_seconds() <= 3600))
    def RecentContent(self, content, updated_at):
        print(f"Rule triggered: Content '{content}' is recent (updated at {updated_at}). Prioritizing.")
        self.declare(Fact(priority='High'))

# Rule 7
    @Rule(Content(content=MATCH.content, category='Unethical'))
    def RejectUnethicalContent(self, content):
        print(f"Rule triggered: Content '{content}' is unethical. Rejecting.")
        self.declare(Fact(stimulus_response='Reject'))

# Rule 8
        # ----------------------------------  Rules about Interaction   --------------------------
    @Rule(Content(content=MATCH.content, interaction=MATCH.interaction, avg_interaction=MATCH.avg_interaction),
        TEST(lambda interaction, avg_interaction: interaction >= 5 * avg_interaction))
    def ReviewHighInteraction(self, content, interaction, avg_interaction):
        print(f"Rule triggered: Content '{content}' has unusually high interaction "
            f"({interaction} vs avg {avg_interaction}). Sending for review.")
        self.declare(Fact(stimulus_response='Review'))


       #---------------------------------------------   FACT      ------------------------------------------------


# Rule 9
    @Rule(Fact(stimulus_situation='Away'),
          Fact(stimulus_response='Check'))
    def Non_publication(self):
        print("Final Rule triggered: Selecting medium 'Non-publication'.")
        self.declare(Medium(medium='Non_publication'))
        print("Recommended Medium: Non-publication\n")

# Rule 10
    @Rule(Fact(priority='High'))
    def PublishImmediately(self):
        print("Final Rule triggered: Content prioritized for immediate publication.")
        self.declare(Medium(medium='Immediate Publication'))
        print("Recommended Medium: Immediate Publication\n")

# Rule 11
    @Rule(Fact(stimulus_response='Reject'))
    def FinalReject(self):
        print("Final Rule triggered: Content marked as 'Rejected'.")
        self.declare(Medium(medium='Rejection'))
        print("Recommended Medium: Rejection\n")
# Rule 12
    @Rule(Fact(stimulus_response='Review'))
    def FinalReview(self):
        print("Final Rule triggered: Content marked for detailed review.")
        self.declare(Medium(medium='Detailed Review'))
        print("Recommended Medium: Detailed Review\n")


x = MediaAdvisor()


Initializing the Media Advisor Expert System...



In [183]:

# قاعدة 1: مصدر موثوق + محتوى وطني + تفاعل طبيعي
print("\nTest Case 1: Credible Source + National Content + Normal Interaction")
x.reset()
x.declare(Credibility(credibility='Old person'))  # مصدر موثوق 100%
x.declare(Content(content='National'))  # محتوى وطني
x.declare(Interaction(interaction=50, avg_interaction=40))  # تفاعل طبيعي
x.run()


Test Case 1: Credible Source + National Content + Normal Interaction
Rule triggered: Content requires 'Check'.
Rule triggered: Credibility is 'Displays'.


In [182]:

# قاعدة 2: مصدر موثوق بنسبة 80% + محتوى مثير للجدل + تفاعل غير طبيعي
print("\nTest Case 2: High Credibility Source + Controversial Content + High Interaction")
x.reset()
x.declare(Credibility(credibility='85'))  # مصدر موثوق بنسبة 85%
x.declare(Content(content='controversial'))  # محتوى مثير للجدل
x.declare(Interaction(interaction=1000, avg_interaction=150))  # تفاعل غير طبيعي (1000 تفاعل)
x.run()


Test Case 2: High Credibility Source + Controversial Content + High Interaction
Rule triggered: Content is 'Controversial'. Notifying the admin.
Rule triggered: High credibility detected.


In [184]:
# قاعدة 3: مصدر غير موثوق + محتوى غير أخلاقي + تفاعل قليل
print("\nTest Case 3: Low Credibility Source + Unethical Content + Low Interaction")
x.reset()
x.declare(Credibility(credibility='Underexcited'))  # مصدر غير موثوق
x.declare(Content(content='Hate Speech', category='Unethical'))  # محتوى غير أخلاقي
x.declare(Interaction(interaction=5, avg_interaction=10))  # تفاعل قليل
x.run()


Test Case 3: Low Credibility Source + Unethical Content + Low Interaction
Rule triggered: Content 'Hate Speech' is unethical. Rejecting.
Final Rule triggered: Content marked as 'Rejected'.
Recommended Medium: Rejection

Rule triggered: Credibility is 'Away'.


In [185]:
# قاعدة 4: محتوى حديث + تفاعل غير طبيعي + مصدر موثوق
print("\nTest Case 4: Recent Content + High Interaction + Credible Source")
x.reset()
recent_time = datetime.now() - timedelta(minutes=30)  # محتوى محدث في آخر 30 دقيقة
x.declare(Credibility(credibility='High'))  # مصدر موثوق بنسبة عالية
x.declare(Content(content='Breaking News', updated_at=recent_time))  # محتوى حديث
x.declare(Interaction(interaction=1000, avg_interaction=150))  # تفاعل غير طبيعي
x.run()


Test Case 4: Recent Content + High Interaction + Credible Source
Rule triggered: Content 'Breaking News' is recent (updated at 2024-12-04 13:13:04.749051). Prioritizing.
Final Rule triggered: Content prioritized for immediate publication.
Recommended Medium: Immediate Publication



In [193]:
# قاعدة 5: محتوى وطني + محتوى مثير للجدل + تفاعل منخفض
print("\nTest Case 5: National Content + Controversial Content + Low Interaction")
x.reset()
x.declare(Content(content='National'))  # محتوى وطني
x.declare(Content(content='controversial'))  # محتوى مثير للجدل
x.declare(Interaction(interaction=5, avg_interaction=10))  # تفاعل منخفض
x.run()


Test Case 5: National Content + Controversial Content + Low Interaction
Rule triggered: Content is 'Controversial'. Notifying the admin.
Rule triggered: Content requires 'Check'.


In [191]:

# قاعدة 6: مصدر غير موثوق + محتوى غير أخلاقي + تفاعل غير طبيعي
print("\nTest Case 6: Low Credibility Source + Unethical Content + High Interaction")
x.reset()
x.declare(Credibility(credibility='Underexcited'))  # مصدر غير موثوق
x.declare(Content(content='Hate Speech', category='Unethical'))  # محتوى غير أخلاقي
x.declare(Interaction(interaction=1000, avg_interaction=200))  # تفاعل غير طبيعي
x.run()


Test Case 6: Low Credibility Source + Unethical Content + High Interaction
Rule triggered: Content 'Hate Speech' is unethical. Rejecting.
Final Rule triggered: Content marked as 'Rejected'.
Recommended Medium: Rejection

Rule triggered: Credibility is 'Away'.


In [194]:

# قاعدة 7: محتوى حديث + محتوى مثير للجدل + تفاعل طبيعي
print("\nTest Case 7: Recent Content + Controversial Content + Normal Interaction")
x.reset()
recent_time = datetime.now() - timedelta(minutes=15)  # محتوى محدث في آخر 15 دقيقة
x.declare(Content(content='Breaking News', updated_at=recent_time))  # محتوى حديث
x.declare(Content(content='controversial'))  # محتوى مثير للجدل
x.declare(Interaction(interaction=50, avg_interaction=40))  # تفاعل طبيعي
x.run()



Test Case 7: Recent Content + Controversial Content + Normal Interaction
Rule triggered: Content is 'Controversial'. Notifying the admin.
Rule triggered: Content 'Breaking News' is recent (updated at 2024-12-04 13:29:21.799161). Prioritizing.
Final Rule triggered: Content prioritized for immediate publication.
Recommended Medium: Immediate Publication

